In [10]:
import os
import laspy
import numpy as np
import pandas as pd
import pickle
from tqdm import tqdm

############################################
# USER CONFIG
############################################

LAS_FOLDER = r"C:/Users/umair.muhammad/Documents/PhD/Research Work/FedLearn/training/All_Nome/las files"
CSV_FOLDER = r"C:/Users/umair.muhammad/Documents/PhD/Research Work/FedLearn/training/All_Nome/txt_labels/cleancsvs/improved_csvs"
OUTPUT_FOLDER = r"C:/Users/umair.muhammad/Documents/PhD/Research Work/FedLearn/training/All_Nome/Full_Pickle"

# Label mapping
CLASS_MAP = {
    "P": 1,       # pothole
    "C": 2,       # crack
    "E": 3,       # edge crack
    "Fl": 4,      # flushed out
}



############################################
# UTILS
############################################

def load_damage_csv(csv_path):
    # Handle missing headers or extra spaces
    df = pd.read_csv(csv_path)
    df.columns = [c.strip() for c in df.columns]  # remove spaces

    boxes = []
    for _, row in df.iterrows():
        boxes.append({
            "bbox": (row["xmin"], row["xmax"], row["ymin"], row["ymax"], row["zmin"], row["zmax"]),
            "type": row["damage_type"].strip()
        })
    return boxes


def label_points(points, boxes):
    labels = np.zeros(points.shape[0], dtype=np.int32)
    for box in boxes:
        xmin, xmax, ymin, ymax, zmin, zmax = box["bbox"]
        cls = CLASS_MAP.get(box["type"], 0)

        mask = (
            (points[:, 0] >= xmin) & (points[:, 0] <= xmax) &
            (points[:, 1] >= ymin) & (points[:, 1] <= ymax) &
            (points[:, 2] >= zmin) & (points[:, 2] <= zmax)
        )
        labels[mask] = cls
    return labels

############################################
# MAIN LOOP
############################################

os.makedirs(OUTPUT_FOLDER, exist_ok=True)
pc_id = 0

for las_file in tqdm(os.listdir(LAS_FOLDER)):
    if not las_file.endswith(".las"):
        continue

    las_path = os.path.join(LAS_FOLDER, las_file)
    
    # Match CSV for this LAS file
    site_name = os.path.splitext(las_file)[0]  # e.g., Site1_0
    csv_path = os.path.join(CSV_FOLDER, f"{site_name}_bboxes.csv")

    print(f"\nProcessing {las_file} -> {csv_path}")

    # read LAS
    las = laspy.read(las_path)
    x = las.x
    y = las.y
    z = las.z

    # Use intensity if color missing
    intensity = las.intensity.astype(np.float32)
    rgb = np.stack([intensity, intensity, intensity], axis=1)

    pts = np.vstack([x, y, z]).T
    pts_full = np.hstack([pts, rgb])  # (N, 6)

    # load bounding boxes
    if os.path.exists(csv_path):
        boxes = load_damage_csv(csv_path)
        print(f"Found {len(boxes)} bounding boxes")
    else:
        boxes = []
        print("No CSV found for this LAS file.")

    # assign labels
    labels = label_points(pts, boxes)
    labels = labels.reshape(-1, 1)

    # final (N,7) format
    final_pc = np.hstack([pts_full, labels])

    # save in required directory structure
    pc_id += 1
    out_dir = os.path.join(OUTPUT_FOLDER, f"pc_id={pc_id}")
    os.makedirs(out_dir, exist_ok=True)
    os.makedirs(os.path.join(out_dir, "metadata"), exist_ok=True)

    # save pc
    with open(os.path.join(out_dir, "pc.pickle"), "wb") as f:
        pickle.dump(final_pc.astype(np.float32), f)

    # save metadata
    metadata = {
        "num_points": final_pc.shape[0],
        "file_source": las_file,
        "classes": CLASS_MAP
    }
    with open(os.path.join(out_dir, "metadata/metadata.pickle"), "wb") as f:
        pickle.dump(metadata, f)

    # print summary
    unique, counts = np.unique(labels, return_counts=True)
    print(f"pc_id={pc_id}: {len(labels)} points")
    for u, c in zip(unique, counts):
        print(f"  class {u}: {c} points")

print("\n✔ Conversion complete!")

  0%|          | 0/185 [00:00<?, ?it/s]


Processing Site1_0.las -> C:/Users/umair.muhammad/Documents/PhD/Research Work/FedLearn/training/All_Nome/txt_labels/cleancsvs/improved_csvs\Site1_0_bboxes.csv
No CSV found for this LAS file.


  2%|▏         | 4/185 [00:00<00:17, 10.59it/s]

pc_id=1: 1917694 points
  class 0: 1917694 points

Processing Site1_1.las -> C:/Users/umair.muhammad/Documents/PhD/Research Work/FedLearn/training/All_Nome/txt_labels/cleancsvs/improved_csvs\Site1_1_bboxes.csv
No CSV found for this LAS file.
pc_id=2: 306546 points
  class 0: 306546 points

Processing Site1_10.las -> C:/Users/umair.muhammad/Documents/PhD/Research Work/FedLearn/training/All_Nome/txt_labels/cleancsvs/improved_csvs\Site1_10_bboxes.csv
No CSV found for this LAS file.
pc_id=3: 497676 points
  class 0: 497676 points

Processing Site1_11.las -> C:/Users/umair.muhammad/Documents/PhD/Research Work/FedLearn/training/All_Nome/txt_labels/cleancsvs/improved_csvs\Site1_11_bboxes.csv
No CSV found for this LAS file.
pc_id=4: 455059 points
  class 0: 455059 points

Processing Site1_12.las -> C:/Users/umair.muhammad/Documents/PhD/Research Work/FedLearn/training/All_Nome/txt_labels/cleancsvs/improved_csvs\Site1_12_bboxes.csv
No CSV found for this LAS file.
pc_id=5: 456490 points
  class 0

  5%|▍         | 9/185 [00:00<00:10, 16.81it/s]

...

No CSV found for this LAS file.
pc_id=183: 1524331 points
  class 0: 1524331 points

Processing SiteV2_8.las -> C:/Users/umair.muhammad/Documents/PhD/Research Work/FedLearn/training/All_Nome/txt_labels/cleancsvs/improved_csvs\SiteV2_8_bboxes.csv


 99%|█████████▉| 184/185 [00:31<00:00,  3.74it/s]

No CSV found for this LAS file.
pc_id=184: 4004789 points
  class 0: 4004789 points

Processing SiteV2_9.las -> C:/Users/umair.muhammad/Documents/PhD/Research Work/FedLearn/training/All_Nome/txt_labels/cleancsvs/improved_csvs\SiteV2_9_bboxes.csv
No CSV found for this LAS file.


100%|██████████| 185/185 [00:31<00:00,  5.79it/s]

pc_id=185: 4972295 points
  class 0: 4972295 points

✔ Conversion complete!


In [2]:
import pickle
import numpy as np

pc_path = r"C:/Users/umair.muhammad/Documents/PhD/Research Work/FedLearn/training/All_Nome/Pickle/pc_id=2/pc.pickle"   # change path

with open(pc_path, "rb") as f:
    pc = pickle.load(f)

print("Type:", type(pc))
print("Shape:", pc.shape)
print("First 5 points:\n", pc[:5])

Type: <class 'numpy.ndarray'>
Shape: (306546, 7)
First 5 points:
 [[4.5262294e+05 7.1835475e+06 1.4552000e+02 1.2000000e+01 1.2000000e+01
  1.2000000e+01 0.0000000e+00]
 [4.5262262e+05 7.1835475e+06 1.4550999e+02 1.1000000e+01 1.1000000e+01
  1.1000000e+01 0.0000000e+00]
 [4.5262231e+05 7.1835470e+06 1.4548000e+02 1.0000000e+01 1.0000000e+01
  1.0000000e+01 0.0000000e+00]
 [4.5262359e+05 7.1835475e+06 1.4552000e+02 2.0000000e+01 2.0000000e+01
  2.0000000e+01 0.0000000e+00]
 [4.5262325e+05 7.1835475e+06 1.4552000e+02 1.9000000e+01 1.9000000e+01
  1.9000000e+01 0.0000000e+00]]


In [15]:
import os
import laspy
import numpy as np
import pandas as pd
import pickle
from tqdm import tqdm
from sklearn.cluster import DBSCAN

############################################
# USER CONFIG
############################################

LAS_FOLDER = r"C:/Users/umair.muhammad/Documents/PhD/Research Work/FedLearn/training/All_Nome/LAS_Files"
CSV_FOLDER = r"C:/Users/umair.muhammad/Documents/PhD/Research Work/FedLearn/training/All_Nome/txt_lables/cleancsvs/improved_csvs"
OUTPUT_FOLDER = r"C:/Users/umair.muhammad/Documents/PhD/Research Work/FedLearn/training/All_Nome/Full_Pickle_dataset"

# Label mapping
CLASS_MAP = {
    "P": 1,       # pothole
    "C": 2,       # crack
    "E": 3,       # edge crack
    "Fl": 4,      # flushed out
}

# Bounding box margin (meters)
BOX_MARGIN = 0.1

# DBSCAN expansion (meters)
USE_DBSCAN = True
DBSCAN_EPS = 0.2
DBSCAN_MIN_SAMPLES = 1

############################################
# UTILS
############################################

def load_damage_csv(csv_path):
    df = pd.read_csv(csv_path)
    df.columns = [c.strip() for c in df.columns]  # remove spaces
    boxes = []
    for _, row in df.iterrows():
        boxes.append({
            "bbox": (row["xmin"], row["xmax"], row["ymin"], row["ymax"], row["zmin"], row["zmax"]),
            "type": row["damage_type"].strip()
        })
    return boxes

def label_points(points, boxes, use_dbscan=True):
    labels = np.zeros(points.shape[0], dtype=np.int32)
    for box in boxes:
        xmin, xmax, ymin, ymax, zmin, zmax = box["bbox"]
        cls = CLASS_MAP.get(box["type"], 0)

        # Inflate bounding box
        mask = (
            (points[:, 0] >= xmin-BOX_MARGIN) & (points[:, 0] <= xmax+BOX_MARGIN) &
            (points[:, 1] >= ymin-BOX_MARGIN) & (points[:, 1] <= ymax+BOX_MARGIN) &
            (points[:, 2] >= zmin-BOX_MARGIN) & (points[:, 2] <= zmax+BOX_MARGIN)
        )
        labels[mask] = cls

    if use_dbscan:
        for cls_val in np.unique(labels):
            if cls_val == 0:
                continue
            idx = np.where(labels == cls_val)[0]
            if len(idx) == 0:
                continue
            coords = points[idx, :3]
            db = DBSCAN(eps=DBSCAN_EPS, min_samples=DBSCAN_MIN_SAMPLES).fit(coords)
            expanded_idx = idx[db.core_sample_indices_]
            labels[expanded_idx] = cls_val

    return labels

############################################
# MAIN LOOP
############################################

os.makedirs(OUTPUT_FOLDER, exist_ok=True)
pc_id = 0

# Summary counters
total_class_counts = {cls: 0 for cls in CLASS_MAP.values()}
total_class_counts[0] = 0  # for unlabeled points

for las_file in tqdm(os.listdir(LAS_FOLDER)):
    if not las_file.endswith(".las"):
        continue

    las_path = os.path.join(LAS_FOLDER, las_file)
    
    # Match CSV for this LAS file
    site_name = os.path.splitext(las_file)[0]  # e.g., Site1_0
    csv_path = os.path.join(CSV_FOLDER, f"{site_name}_bboxes.csv")

    print(f"\nProcessing {las_file} -> {csv_path}")

    # read LAS
    las = laspy.read(las_path)
    x = las.x
    y = las.y
    z = las.z

    # Use intensity as feature (or RGB if available)
    intensity = las.intensity.astype(np.float32)
    rgb = np.stack([intensity, intensity, intensity], axis=1)

    pts = np.vstack([x, y, z]).T
    pts_full = np.hstack([pts, rgb])  # (N, 6)

    # load bounding boxes
    if os.path.exists(csv_path):
        boxes = load_damage_csv(csv_path)
        print(f"Found {len(boxes)} bounding boxes")
    else:
        boxes = []
        print("No CSV found for this LAS file.")

    # assign labels with DBSCAN expansion
    labels = label_points(pts, boxes, use_dbscan=USE_DBSCAN)
    labels = labels.reshape(-1, 1)

    # Update summary counts
    unique, counts = np.unique(labels, return_counts=True)
    for u, c in zip(unique, counts):
        total_class_counts[u] += c

    # final (N,7) format
    final_pc = np.hstack([pts_full, labels])

    # save in required directory structure
    pc_id += 1
    out_dir = os.path.join(OUTPUT_FOLDER, f"pc_id={pc_id}")
    os.makedirs(out_dir, exist_ok=True)
    os.makedirs(os.path.join(out_dir, "metadata"), exist_ok=True)

    # save pc
    with open(os.path.join(out_dir, "pc.pickle"), "wb") as f:
        pickle.dump(final_pc.astype(np.float32), f)

    # save metadata
    metadata = {
        "num_points": final_pc.shape[0],
        "file_source": las_file,
        "classes": CLASS_MAP
    }
    with open(os.path.join(out_dir, "metadata/metadata.pickle"), "wb") as f:
        pickle.dump(metadata, f)

    # print per-file summary
    print(f"pc_id={pc_id}: {len(labels)} points")
    for u, c in zip(unique, counts):
        print(f"  class {u}: {c} points")

# ------------------------------
# Final summary
# ------------------------------
print("\n✔ Conversion complete!")
print("\n==== TOTAL POINTS PER CLASS ACROSS ALL FILES ====")
for cls, count in total_class_counts.items():
    cls_name = "unlabeled" if cls == 0 else [k for k,v in CLASS_MAP.items() if v==cls][0]
    print(f"Class {cls} ({cls_name}): {count} points")


  0%|          | 0/149 [00:00<?, ?it/s]


Processing Site1_0.las -> C:/Users/umair.muhammad/Documents/PhD/Research Work/FedLearn/training/All_Nome/txt_lables/cleancsvs/improved_csvs\Site1_0_bboxes.csv
No CSV found for this LAS file.


  3%|▎         | 4/149 [00:00<00:12, 11.46it/s]

pc_id=1: 1917694 points
  class 0: 1917694 points

Processing Site1_1.las -> C:/Users/umair.muhammad/Documents/PhD/Research Work/FedLearn/training/All_Nome/txt_lables/cleancsvs/improved_csvs\Site1_1_bboxes.csv
No CSV found for this LAS file.
pc_id=2: 306546 points
  class 0: 306546 points

Processing Site1_10.las -> C:/Users/umair.muhammad/Documents/PhD/Research Work/FedLearn/training/All_Nome/txt_lables/cleancsvs/improved_csvs\Site1_10_bboxes.csv
No CSV found for this LAS file.
pc_id=3: 497676 points
  class 0: 497676 points

Processing Site1_11.las -> C:/Users/umair.muhammad/Documents/PhD/Research Work/FedLearn/training/All_Nome/txt_lables/cleancsvs/improved_csvs\Site1_11_bboxes.csv
No CSV found for this LAS file.
pc_id=4: 455059 points
  class 0: 455059 points

Processing Site1_12.las -> C:/Users/umair.muhammad/Documents/PhD/Research Work/FedLearn/training/All_Nome/txt_lables/cleancsvs/improved_csvs\Site1_12_bboxes.csv
No CSV found for this LAS file.
pc_id=5: 456490 points
  class 0

  6%|▌         | 9/149 [00:00<00:08, 17.11it/s]

...

pc_id=147: 2732103 points
  class 0: 2732103 points

Processing SiteV2_8.las -> C:/Users/umair.muhammad/Documents/PhD/Research Work/FedLearn/training/All_Nome/txt_lables/cleancsvs/improved_csvs\SiteV2_8_bboxes.csv


 99%|█████████▉| 148/149 [00:26<00:00,  3.26it/s]

No CSV found for this LAS file.
pc_id=148: 4004789 points
  class 0: 4004789 points

Processing SiteV2_9.las -> C:/Users/umair.muhammad/Documents/PhD/Research Work/FedLearn/training/All_Nome/txt_lables/cleancsvs/improved_csvs\SiteV2_9_bboxes.csv
No CSV found for this LAS file.


100%|██████████| 149/149 [00:26<00:00,  5.56it/s]

pc_id=149: 4972295 points
  class 0: 4972295 points

✔ Conversion complete!

==== TOTAL POINTS PER CLASS ACROSS ALL FILES ====
Class 1 (P): 0 points
Class 2 (C): 0 points
Class 3 (E): 0 points
Class 4 (Fl): 0 points
Class 0 (unlabeled): 239911483 points


In [ ]:
import os
import laspy
import numpy as np
import pandas as pd
import pickle
from tqdm import tqdm
from sklearn.cluster import DBSCAN

############################################
# USER CONFIG
############################################

LAS_FOLDER = "C:/Users/umair.muhammad/Documents/PhD/Research Work/FedLearn/training/All_Nome/LAS_Files"
CSV_FOLDER = "C:/Users/umair.muhammad/Documents/PhD/Research Work/FedLearn/training/All_Nome/txt_lables/cleancsvs/improved_csvs"
OUTPUT_FOLDER = "C:/Users/umair.muhammad/Documents/PhD/Research Work/FedLearn/training/All_Nome/Full_Pickle_dataset"

# Label mapping
CLASS_MAP = {
    "P": 1,       # pothole
    "C": 2,       # crack
    "E": 3,       # edge crack
    "Fl": 4,      # flushed out
}

# Bounding box margin (meters)
BOX_MARGIN = 0.1

# DBSCAN expansion (meters)
USE_DBSCAN = True
DBSCAN_EPS = 0.2
DBSCAN_MIN_SAMPLES = 1

############################################
# UTILS
############################################

def load_damage_csv(csv_path):
    """Load bounding boxes from CSV"""
    df = pd.read_csv(csv_path)
    df.columns = [c.strip() for c in df.columns]  # remove spaces
    boxes = []
    for _, row in df.iterrows():
        boxes.append({
            "bbox": (row["Xmin"], row["Xmax"], row["Ymin"], row["Ymax"], row["Zmin"], row["Zmax"]),
            "type": row["Damage_Type"].strip()
        })
    return boxes

def label_points(points, boxes, use_dbscan=True):
    """Assign class labels to points given bounding boxes"""
    labels = np.zeros(points.shape[0], dtype=np.int32)
    for box in boxes:
        xmin, xmax, ymin, ymax, zmin, zmax = box["bbox"]
        cls = CLASS_MAP.get(box["type"], 0)

        # Inflate bounding box
        mask = (
            (points[:, 0] >= xmin - BOX_MARGIN) & (points[:, 0] <= xmax + BOX_MARGIN) &
            (points[:, 1] >= ymin - BOX_MARGIN) & (points[:, 1] <= ymax + BOX_MARGIN) &
            (points[:, 2] >= zmin - BOX_MARGIN) & (points[:, 2] <= zmax + BOX_MARGIN)
        )
        labels[mask] = cls

    # Optional DBSCAN expansion
    if use_dbscan:
        for cls_val in np.unique(labels):
            if cls_val == 0:
                continue
            idx = np.where(labels == cls_val)[0]
            if len(idx) == 0:
                continue
            coords = points[idx, :3]
            db = DBSCAN(eps=DBSCAN_EPS, min_samples=DBSCAN_MIN_SAMPLES).fit(coords)
            if hasattr(db, 'core_sample_indices_') and len(db.core_sample_indices_) > 0:
                expanded_idx = idx[db.core_sample_indices_]
                labels[expanded_idx] = cls_val

    return labels

############################################
# MAIN LOOP
############################################

os.makedirs(OUTPUT_FOLDER, exist_ok=True)
pc_id = 0

# Summary counters
total_class_counts = {cls: 0 for cls in CLASS_MAP.values()}
total_class_counts[0] = 0  # for unlabeled points

for las_file in tqdm(os.listdir(LAS_FOLDER)):
    if not las_file.lower().endswith(".las"):
        continue

    las_path = os.path.join(LAS_FOLDER, las_file)
    
    # Match CSV for this LAS file
    site_name = os.path.splitext(las_file)[0]  # e.g., Site1_7
    csv_path = os.path.join(CSV_FOLDER, f"{site_name}.csv")

    if not os.path.exists(csv_path):
        print(f"\nSkipping {las_file}: No CSV found -> {csv_path}")
        continue  # skip this LAS file entirely

    print(f"\nProcessing {las_file} -> {csv_path}")

    # read LAS
    las = laspy.read(las_path)
    x = las.x
    y = las.y
    z = las.z

    # Use intensity as feature (or RGB if available)
    intensity = las.intensity.astype(np.float32)
    rgb = np.stack([intensity, intensity, intensity], axis=1)

    pts = np.vstack([x, y, z]).T
    pts_full = np.hstack([pts, rgb])  # (N, 6)

    # load bounding boxes
    boxes = load_damage_csv(csv_path)
    print(f"Found {len(boxes)} bounding boxes")

    # assign labels with DBSCAN expansion
    labels = label_points(pts, boxes, use_dbscan=USE_DBSCAN)
    labels = labels.reshape(-1, 1)

    # Update summary counts
    unique, counts = np.unique(labels, return_counts=True)
    for u, c in zip(unique, counts):
        total_class_counts[u] += c

    # final (N,7) format
    final_pc = np.hstack([pts_full, labels])

    # save in required directory structure
    pc_id += 1
    out_dir = os.path.join(OUTPUT_FOLDER, f"pc_id={pc_id}")
    os.makedirs(out_dir, exist_ok=True)
    os.makedirs(os.path.join(out_dir, "metadata"), exist_ok=True)

    # save pc
    with open(os.path.join(out_dir, "pc.pickle"), "wb") as f:
        pickle.dump(final_pc.astype(np.float32), f)

    # save metadata
    metadata = {
        "num_points": final_pc.shape[0],
        "file_source": las_file,
        "classes": CLASS_MAP
    }
    with open(os.path.join(out_dir, "metadata/metadata.pickle"), "wb") as f:
        pickle.dump(metadata, f)

    # print per-file summary
    print(f"pc_id={pc_id}: {len(labels)} points")
    for u, c in zip(unique, counts):
        cls_name = "unlabeled" if u == 0 else [k for k,v in CLASS_MAP.items() if v==u][0]
        print(f"  class {u} ({cls_name}): {c} points")

# ------------------------------
# Final summary
# ------------------------------
print("\n✔ Conversion complete!")
print("\n==== TOTAL POINTS PER CLASS ACROSS ALL FILES ====")
print(f"{'Class':<10}{'Label':<10}{'Points':<15}")
print("-" * 35)
for cls, count in total_class_counts.items():
    cls_name = "unlabeled" if cls == 0 else [k for k,v in CLASS_MAP.items() if v==cls][0]
    print(f"{cls:<10}{cls_name:<10}{count:<15}")

total_points = sum(total_class_counts.values())
print(f"\nTotal points across all processed LAS files: {total_points}")


  0%|          | 0/149 [00:00<?, ?it/s]


Processing Site1_0.las -> C:/Users/umair.muhammad/Documents/PhD/Research Work/FedLearn/training/All_Nome/txt_lables/cleancsvs/improved_csvs\Site1_0.csv
Found 1 bounding boxes


  2%|▏         | 3/149 [00:00<00:20,  7.23it/s]

pc_id=1: 1917694 points
  class 0 (unlabeled): 1917215 points
  class 1 (P): 479 points

Processing Site1_1.las -> C:/Users/umair.muhammad/Documents/PhD/Research Work/FedLearn/training/All_Nome/txt_lables/cleancsvs/improved_csvs\Site1_1.csv
Found 20 bounding boxes
pc_id=2: 306546 points
  class 0 (unlabeled): 303805 points
  class 1 (P): 2741 points

Processing Site1_10.las -> C:/Users/umair.muhammad/Documents/PhD/Research Work/FedLearn/training/All_Nome/txt_lables/cleancsvs/improved_csvs\Site1_10.csv
Found 38 bounding boxes
pc_id=3: 497676 points
  class 0 (unlabeled): 484876 points
  class 1 (P): 12800 points


  3%|▎         | 4/149 [00:00<00:18,  7.90it/s]

...

pc_id=122: 1847245 points
  class 0 (unlabeled): 1846520 points
  class 1 (P): 725 points

Processing SiteV2_48.las -> C:/Users/umair.muhammad/Documents/PhD/Research Work/FedLearn/training/All_Nome/txt_lables/cleancsvs/improved_csvs\SiteV2_48.csv
Found 38 bounding boxes


 85%|████████▍ | 126/149 [00:40<00:19,  1.20it/s]

pc_id=123: 2213586 points
  class 0 (unlabeled): 2210229 points
  class 1 (P): 3357 points

Processing SiteV2_5.las -> C:/Users/umair.muhammad/Documents/PhD/Research Work/FedLearn/training/All_Nome/txt_lables/cleancsvs/improved_csvs\SiteV2_5.csv
Found 130 bounding boxes


 85%|████████▌ | 127/149 [00:41<00:24,  1.11s/it]

pc_id=124: 2853130 points
  class 0 (unlabeled): 2840626 points
  class 1 (P): 12504 points

Processing SiteV2_50.las -> C:/Users/umair.muhammad/Documents/PhD/Research Work/FedLearn/training/All_Nome/txt_lables/cleancsvs/improved_csvs\SiteV2_50.csv
Found 128 bounding boxes


In [20]:
# ------------------------------
# Final summary
# ------------------------------
print("\n✔ Conversion complete!")
print("\n==== TOTAL POINTS PER CLASS ACROSS ALL FILES ====")
print(f"{'Class':<10}{'Label':<10}{'Points':<15}")
print("-" * 35)
for cls, count in total_class_counts.items():
    cls_name = "unlabeled" if cls == 0 else [k for k,v in CLASS_MAP.items() if v==cls][0]
    print(f"{cls:<10}{cls_name:<10}{count:<15}")



✔ Conversion complete!

==== TOTAL POINTS PER CLASS ACROSS ALL FILES ====
Class     Label     Points         
-----------------------------------
1         P         596104         
2         C         108673         
3         E         6276           
4         Fl        8295           
0         unlabeled 239192135      


In [8]:
import os
import laspy
import numpy as np
import pandas as pd
import pickle
from tqdm import tqdm
from sklearn.cluster import DBSCAN

############################################
# USER CONFIG
############################################

LAS_FOLDER = r"C:/Users/umair.muhammad/Documents/PhD/Research Work/FedLearn/training/All_Nome/LAS_Files_Site1"
CSV_FOLDER = r"C:/Users/umair.muhammad/Documents/PhD/Research Work/FedLearn/training/All_Nome/Labels_Site1/save_bboxes"
OUTPUT_FOLDER = r"C:/Users/umair.muhammad/Documents/PhD/Research Work/FedLearn/training/All_Nome/Pickle"

# Label mapping
CLASS_MAP = {
    "P": 1,       # pothole
    "C": 2,       # crack
    "E": 3,       # edge crack
    "Fl": 4,      # flushed out
}

# Bounding box margin and DBSCAN parameters
BOX_MARGIN = 0.2
DBSCAN_EPS = 0.3
DBSCAN_MIN_SAMPLES = 1

############################################
# UTILS
############################################

def load_damage_csv(csv_path):
    df = pd.read_csv(csv_path)
    df.columns = [c.strip() for c in df.columns]  # remove spaces
    boxes = []
    for _, row in df.iterrows():
        boxes.append({
            "bbox": (row["xmin"], row["xmax"], row["ymin"], row["ymax"], row["zmin"], row["zmax"]),
            "type": row["damage_type"].strip()
        })
    return boxes

def label_points(points, boxes, use_dbscan=True):
    labels = np.zeros(points.shape[0], dtype=np.int32)

    for box in boxes:
        xmin, xmax, ymin, ymax, zmin, zmax = box["bbox"]
        cls = CLASS_MAP.get(box["type"], 0)

        # Inflate bounding box
        mask = (
            (points[:, 0] >= xmin - BOX_MARGIN) & (points[:, 0] <= xmax + BOX_MARGIN) &
            (points[:, 1] >= ymin - BOX_MARGIN) & (points[:, 1] <= ymax + BOX_MARGIN) &
            (points[:, 2] >= zmin - BOX_MARGIN) & (points[:, 2] <= zmax + BOX_MARGIN)
        )
        labels[mask] = cls

    # DBSCAN only for potholes (class 1)
    if use_dbscan:
        cls_val = 1
        idx = np.where(labels == cls_val)[0]
        if len(idx) > 0:
            coords = points[idx, :3]
            db = DBSCAN(eps=DBSCAN_EPS, min_samples=DBSCAN_MIN_SAMPLES).fit(coords)
            expanded_idx = idx[db.core_sample_indices_]
            labels[expanded_idx] = cls_val

    return labels

############################################
# MAIN LOOP
############################################

os.makedirs(OUTPUT_FOLDER, exist_ok=True)
pc_id = 0
total_counts = {k: 0 for k in CLASS_MAP.values()}
total_counts[0] = 0  # background

for las_file in tqdm(os.listdir(LAS_FOLDER)):
    if not las_file.endswith(".las"):
        continue

    las_path = os.path.join(LAS_FOLDER, las_file)
    
    site_name = os.path.splitext(las_file)[0]
    csv_path = os.path.join(CSV_FOLDER, f"{site_name}_bboxes.csv")

    print(f"\nProcessing {las_file} -> {csv_path}")

    # Read LAS
    las = laspy.read(las_path)
    x, y, z = las.x, las.y, las.z
    intensity = las.intensity.astype(np.float32)
    rgb = np.stack([intensity, intensity, intensity], axis=1)

    pts = np.vstack([x, y, z]).T
    pts_full = np.hstack([pts, rgb])  # (N,6)

    # Load bounding boxes
    if os.path.exists(csv_path):
        boxes = load_damage_csv(csv_path)
        print(f"Found {len(boxes)} bounding boxes")
    else:
        boxes = []
        print("No CSV found for this LAS file.")

    # Assign labels
    labels = label_points(pts, boxes, use_dbscan=True)
    labels = labels.reshape(-1,1)
    final_pc = np.hstack([pts_full, labels])  # (N,7)

    # Save pc
    pc_id += 1
    out_dir = os.path.join(OUTPUT_FOLDER, f"pc_id={pc_id}")
    os.makedirs(out_dir, exist_ok=True)
    os.makedirs(os.path.join(out_dir, "metadata"), exist_ok=True)
    with open(os.path.join(out_dir, "pc.pickle"), "wb") as f:
        pickle.dump(final_pc.astype(np.float32), f)

    # Save metadata
    metadata = {
        "num_points": final_pc.shape[0],
        "file_source": las_file,
        "classes": CLASS_MAP
    }
    with open(os.path.join(out_dir, "metadata/metadata.pickle"), "wb") as f:
        pickle.dump(metadata, f)

    # Print per-file summary
    unique, counts = np.unique(labels, return_counts=True)
    print(f"pc_id={pc_id}: {len(labels)} points")
    for u, c in zip(unique, counts):
        print(f"  class {u}: {c} points")
        total_counts[u] += c

# Print total summary
print("\n✔ Conversion complete! Total points per class:")
for cls, c in total_counts.items():
    print(f"  class {cls}: {c} points")


  7%|▋         | 1/15 [00:00<00:02,  5.19it/s]


Processing Site1_0.las -> C:/Users/umair.muhammad/Documents/PhD/Research Work/FedLearn/training/All_Nome/Labels_Site1/save_bboxes\Site1_0_bboxes.csv
Found 1 bounding boxes
pc_id=1: 1917694 points
  class 0: 1916973 points
  class 1: 721 points

Processing Site1_1.las -> C:/Users/umair.muhammad/Documents/PhD/Research Work/FedLearn/training/All_Nome/Labels_Site1/save_bboxes\Site1_1_bboxes.csv
Found 20 bounding boxes
pc_id=2: 306546 points
  class 0: 301376 points
  class 1: 5170 points

Processing Site1_10.las -> C:/Users/umair.muhammad/Documents/PhD/Research Work/FedLearn/training/All_Nome/Labels_Site1/save_bboxes\Site1_10_bboxes.csv
Found 40 bounding boxes


 27%|██▋       | 4/15 [00:00<00:01,  6.16it/s]

pc_id=3: 497676 points
  class 0: 474477 points
  class 1: 23199 points

Processing Site1_11.las -> C:/Users/umair.muhammad/Documents/PhD/Research Work/FedLearn/training/All_Nome/Labels_Site1/save_bboxes\Site1_11_bboxes.csv
Found 31 bounding boxes
pc_id=4: 455059 points
  class 0: 437803 points
  class 1: 15659 points
  class 2: 1597 points

Processing Site1_12.las -> C:/Users/umair.muhammad/Documents/PhD/Research Work/FedLearn/training/All_Nome/Labels_Site1/save_bboxes\Site1_12_bboxes.csv
Found 35 bounding boxes


 40%|████      | 6/15 [00:01<00:01,  6.01it/s]

...

pc_id=7: 415143 points
  class 0: 379795 points
  class 1: 17930 points
  class 2: 17418 points

Processing Site1_2.las -> C:/Users/umair.muhammad/Documents/PhD/Research Work/FedLearn/training/All_Nome/Labels_Site1/save_bboxes\Site1_2_bboxes.csv
Found 11 bounding boxes
pc_id=8: 356774 points
  class 0: 352385 points
  class 1: 4389 points

Processing Site1_3.las -> C:/Users/umair.muhammad/Documents/PhD/Research Work/FedLearn/training/All_Nome/Labels_Site1/save_bboxes\Site1_3_bboxes.csv
Found 18 bounding boxes
pc_id=9: 373677 points
  class 0: 366889 points
  class 1: 6788 points

Processing Site1_4.las -> C:/Users/umair.muhammad/Documents/PhD/Research Work/FedLearn/training/All_Nome/Labels_Site1/save_bboxes\Site1_4_bboxes.csv
Found 12 bounding boxes
pc_id=10: 336539 points
  class 0: 313327 points
  class 1: 4115 points
  class 2: 19097 points

Processing Site1_5.las -> C:/Users/umair.muhammad/Documents/PhD/Research Work/FedLearn/training/All_Nome/Labels_Site1/save_bboxes\Site1_5_bboxe

 93%|█████████▎| 14/15 [00:01<00:00, 13.05it/s]

Found 10 bounding boxes
pc_id=11: 315342 points
  class 0: 307458 points
  class 1: 3210 points
  class 2: 1236 points
  class 3: 3438 points

Processing Site1_6.las -> C:/Users/umair.muhammad/Documents/PhD/Research Work/FedLearn/training/All_Nome/Labels_Site1/save_bboxes\Site1_6_bboxes.csv
Found 3 bounding boxes
pc_id=12: 254686 points
  class 0: 252536 points
  class 1: 1042 points
  class 3: 1108 points

Processing Site1_7.las -> C:/Users/umair.muhammad/Documents/PhD/Research Work/FedLearn/training/All_Nome/Labels_Site1/save_bboxes\Site1_7_bboxes.csv
Found 18 bounding boxes
pc_id=13: 300495 points
  class 0: 285990 points
  class 1: 5798 points
  class 2: 8707 points

Processing Site1_8.las -> C:/Users/umair.muhammad/Documents/PhD/Research Work/FedLearn/training/All_Nome/Labels_Site1/save_bboxes\Site1_8_bboxes.csv
Found 9 bounding boxes
pc_id=14: 438942 points
  class 0: 390299 points
  class 1: 4626 points
  class 2: 44017 points


100%|██████████| 15/15 [00:01<00:00,  8.50it/s]


Processing Site1_9.las -> C:/Users/umair.muhammad/Documents/PhD/Research Work/FedLearn/training/All_Nome/Labels_Site1/save_bboxes\Site1_9_bboxes.csv
Found 24 bounding boxes
pc_id=15: 439928 points
  class 0: 403457 points
  class 1: 14383 points
  class 2: 22088 points

✔ Conversion complete! Total points per class:
  class 1: 134710 points
  class 2: 114160 points
  class 3: 4546 points
  class 4: 0 points
  class 0: 7310253 points
